In [70]:
import pandas as pd
import numpy as np
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
from snowflake.snowpark.files import SnowflakeFile
import sys
sys.path.append('..')
from credentials import Credentials

In [71]:
# Setting pandas options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [72]:
cred = Credentials()
session = Session.builder.configs(cred.__dict__).create()

In [73]:
session.use_role("SYSADMIN")
session.use_database("STITCH")
session.use_warehouse("ACCOUNTADMIN_MGMT")
session.use_schema("SALESFORCEFSL3")

# Indiviual Table 

In [74]:
# Getting name of the table
db_name = 'STITCH'
schema_name = 'SALESFORCEFSL3'
table_name:str = 'ACCOUNT'

In [75]:
result_describe_table = [row.as_dict() for row in session.sql(f"DESCRIBE TABLE {db_name}.{schema_name}.{table_name};".format(db_name, schema_name, table_name)).collect()]

In [76]:
df_describe_table = pd.DataFrame(result_describe_table)

In [77]:
df_describe_table.head(10)

,name,type,kind,null?,default,primary key,unique key,check,expression,comment,policy name
0,ABM_TARGET_ACCOUNT__C,BOOLEAN,COLUMN,Y,None,N,N,None,None,None,None
1,ACCOUNTNUMBER,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None,None
2,ACCOUNT_ISACTIVE_CURRENT_FY__C,BOOLEAN,COLUMN,Y,None,N,N,None,None,None,None
3,ACCOUNT_ISACTIVE_CURRENT_PREVIOUS_FY__C,BOOLEAN,COLUMN,Y,None,N,N,None,None,None,None
4,ACCOUNT_ISACTIVE_LAST_12_MONTHS__C,BOOLEAN,COLUMN,Y,None,N,N,None,None,None,None
5,ACCOUNT_OWNER_ROLE__C,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None,None
6,ACCOUNT_PLAN_COUNT__C,FLOAT,COLUMN,Y,None,N,N,None,None,None,None
7,ACCOUNT_RECORD_ID__C,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None,None
8,ACCOUNT_REGION__C,FLOAT,COLUMN,Y,None,N,N,None,None,None,None
9,AM_IS_ME__C,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None,None


## Mixing with Mapping table

In [78]:
result_salesforce_mapping = [row.as_dict() for row in session.sql(f"SELECT SALESFORCE_ATTRIBUTE_NAME, NEW_NAME FROM ACCOUNTADMIN_MGMT.UTILITIES.SALESFORCE_MAPPING WHERE SALESFORCE_OBJECT_NAME = '{table_name}'".format()).collect()]
df_salesforce_mapping = pd.DataFrame(result_salesforce_mapping)

In [79]:
df_salesforce_mapping

""


In [80]:
if df_salesforce_mapping.empty:
	final_df = df_describe_table
else:
	final_df = pd.merge(df_describe_table, df_salesforce_mapping, left_on='name', right_on='SALESFORCE_ATTRIBUTE_NAME', how='left')
	final_df['NEW_NAME'].fillna('MISSING_VALUE_PLACEHOLDER', inplace=True)
	final_df['name'] = final_df.apply(lambda x: x['name'] if x['NEW_NAME'] == 'MISSING_VALUE_PLACEHOLDER' else x['NEW_NAME'], axis=1)
	final_df.drop(['SALESFORCE_ATTRIBUTE_NAME', 'NEW_NAME'], axis=1, inplace=True)


In [81]:
final_df.head(10)

,name,type,kind,null?,default,primary key,unique key,check,expression,comment,policy name
0,ABM_TARGET_ACCOUNT__C,BOOLEAN,COLUMN,Y,None,N,N,None,None,None,None
1,ACCOUNTNUMBER,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None,None
2,ACCOUNT_ISACTIVE_CURRENT_FY__C,BOOLEAN,COLUMN,Y,None,N,N,None,None,None,None
3,ACCOUNT_ISACTIVE_CURRENT_PREVIOUS_FY__C,BOOLEAN,COLUMN,Y,None,N,N,None,None,None,None
4,ACCOUNT_ISACTIVE_LAST_12_MONTHS__C,BOOLEAN,COLUMN,Y,None,N,N,None,None,None,None
5,ACCOUNT_OWNER_ROLE__C,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None,None
6,ACCOUNT_PLAN_COUNT__C,FLOAT,COLUMN,Y,None,N,N,None,None,None,None
7,ACCOUNT_RECORD_ID__C,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None,None
8,ACCOUNT_REGION__C,FLOAT,COLUMN,Y,None,N,N,None,None,None,None
9,AM_IS_ME__C,VARCHAR(16777216),COLUMN,Y,None,N,N,None,None,None,None


In [82]:
# a lambda function that creates a new column called 'name_new' if the type is like VARCHAR then add a "000" to the origial name else just add the original name
# df['name_new'] = df.apply(lambda x: x['name'] + '000' if isinstance(x['type'], str) and x['type'].find('VARCHAR') != -1 else x['name'], axis=1)

## Doing transformation

In [83]:
def transform_name(name, type):
    if isinstance(type, str) and 'VARCHAR' in type:
        #return name + '0007'
        return f"CAST(SUBSTR({name},1,10000) AS VARCHAR(10000)) AS {name}".format(name)
    else:
        return f"{name} AS {name}".format(name)

# apply the transform_name function to create a new column called 'name_new'
final_df['name_new'] = final_df.apply(lambda x: transform_name(x['name'], x['type']), axis=1)

In [84]:

#df[['name','type','name_new']].head(200)
column_list:list = final_df['name_new'].to_list()

In [85]:
columns_str:str = ', '.join(column_list)

In [86]:
columns_str

'ABM_TARGET_ACCOUNT__C AS ABM_TARGET_ACCOUNT__C, CAST(SUBSTR(ACCOUNTNUMBER,1,10000) AS VARCHAR(10000)) AS ACCOUNTNUMBER, ACCOUNT_ISACTIVE_CURRENT_FY__C AS ACCOUNT_ISACTIVE_CURRENT_FY__C, ACCOUNT_ISACTIVE_CURRENT_PREVIOUS_FY__C AS ACCOUNT_ISACTIVE_CURRENT_PREVIOUS_FY__C, ACCOUNT_ISACTIVE_LAST_12_MONTHS__C AS ACCOUNT_ISACTIVE_LAST_12_MONTHS__C, CAST(SUBSTR(ACCOUNT_OWNER_ROLE__C,1,10000) AS VARCHAR(10000)) AS ACCOUNT_OWNER_ROLE__C, ACCOUNT_PLAN_COUNT__C AS ACCOUNT_PLAN_COUNT__C, CAST(SUBSTR(ACCOUNT_RECORD_ID__C,1,10000) AS VARCHAR(10000)) AS ACCOUNT_RECORD_ID__C, ACCOUNT_REGION__C AS ACCOUNT_REGION__C, CAST(SUBSTR(AM_IS_ME__C,1,10000) AS VARCHAR(10000)) AS AM_IS_ME__C, CAST(SUBSTR(BILLINGCITY,1,10000) AS VARCHAR(10000)) AS BILLINGCITY, CAST(SUBSTR(BILLINGCOUNTRY,1,10000) AS VARCHAR(10000)) AS BILLINGCOUNTRY, CAST(SUBSTR(BILLINGCOUNTRYCODE,1,10000) AS VARCHAR(10000)) AS BILLINGCOUNTRYCODE, CAST(SUBSTR(BILLINGGEOCODEACCURACY,1,10000) AS VARCHAR(10000)) AS BILLINGGEOCODEACCURACY, BILLINGLATI

In [63]:
# append the final result to something like "CREATE OR REPLACE VIEW {db_name}.{schema_name}}.{table_name}_V AS SELECT {columns_str} FROM {db_name}.{schema_name}.{table_name};"
final_query:str = f"CREATE OR REPLACE VIEW {db_name}.{schema_name}.{table_name}_V AS SELECT {columns_str} FROM {db_name}.{schema_name}.{table_name};"

In [64]:
final_query

'CREATE OR REPLACE VIEW STITCH.SALESFORCEFSL3.ACCOUNT_V AS SELECT ABM_TARGET_ACCOUNT__C AS ABM_TARGET_ACCOUNT__C, CAST(SUBSTR(ACCOUNTNUMBER,1,10000) AS VARCHAR(10000)) AS ACCOUNTNUMBER, ACCOUNT_ISACTIVE_CURRENT_FY__C AS ACCOUNT_ISACTIVE_CURRENT_FY__C, ACCOUNT_ISACTIVE_CURRENT_PREVIOUS_FY__C AS ACCOUNT_ISACTIVE_CURRENT_PREVIOUS_FY__C, ACCOUNT_ISACTIVE_LAST_12_MONTHS__C AS ACCOUNT_ISACTIVE_LAST_12_MONTHS__C, CAST(SUBSTR(ACCOUNT_OWNER_ROLE__C,1,10000) AS VARCHAR(10000)) AS ACCOUNT_OWNER_ROLE__C, ACCOUNT_PLAN_COUNT__C AS ACCOUNT_PLAN_COUNT__C, CAST(SUBSTR(ACCOUNT_RECORD_ID__C,1,10000) AS VARCHAR(10000)) AS ACCOUNT_RECORD_ID__C, ACCOUNT_REGION__C AS ACCOUNT_REGION__C, CAST(SUBSTR(AM_IS_ME__C,1,10000) AS VARCHAR(10000)) AS AM_IS_ME__C, CAST(SUBSTR(BILLINGCITY,1,10000) AS VARCHAR(10000)) AS BILLINGCITY, CAST(SUBSTR(BILLINGCOUNTRY,1,10000) AS VARCHAR(10000)) AS BILLINGCOUNTRY, CAST(SUBSTR(BILLINGCOUNTRYCODE,1,10000) AS VARCHAR(10000)) AS BILLINGCOUNTRYCODE, CAST(SUBSTR(BILLINGGEOCODEACCURACY,1

In [66]:
session.sql(final_query).collect()

[Row(status='View ACCOUNT_V successfully created.')]

# All tables

In [21]:
database:str = 'STITCH'

for schema in ['SALESFORCEFSL3']:
    result = session.sql(f"SHOW TABLES IN STITCH.{schema}".format(schema)).collect()
    #[row.as_dict() for row in result]
    df = pd.DataFrame(result)
    tables:list = df['name'].to_list()
    for table in tables:
        if table != '_SDC_REJECTED':
            print(table)
            #create_view_query:str = f"CALL ACCOUNTADMIN_MGMT.UTILITIES.CREATE_DYNAMIC_SALESFORCE_VIEW('{database}', '{schema}', '{table}');".format(database, schema, table)
            #session.sql(create_view_query).collect()


ACCOUNT
ACCOUNTCONTACTRELATION
ASSET
ASSIGNEDRESOURCE
ATTACHMENT
CASE
CASEARTICLE
CASECOMMENT
CONTACT
CONTRACTLINEITEM
ENTITLEMENT
FSM_AUTO_ENTITLEMENT_MATRIX__C
FSM_BILLING_SCHEDULE__C
FSM_CODE_REPOSITORY__C
FSM_CUSTOMER_MODEL__C
FSM_ENTITLEMENT_SERVICE_ACTIVITY__C
FSM_ESR_CONFIG_MASTER__C
FSM_ESR_INTERFACE_FLAG__C
FSM_ESR_MAPPING__C
FSM_SERVICE_ACTIVITY__C
HOLIDAY
LOCATION
OPERATINGHOURS
OPERATINGHOURSHOLIDAY
PRICEBOOK2
PRICEBOOKENTRY
PRODUCT2
PRODUCTITEM
PRODUCTREQUEST
SERVICEAPPOINTMENT
SERVICECONTRACT
SERVICERESOURCE
SERVICETERRITORY
SERVICETERRITORYMEMBER
WORKORDER
WORKORDERLINEITEM


In [69]:
session.close()